In [2]:
import random

In [3]:
# Key Encoding:
# Total-AceAs11-Action
# (Total of Cards)-(0:No, 1:Yes)-(0:Stand, 1:Hit)

# Dict Value:
# [Counts, Probability]
class stateActionVal:
    def __init__(self):
        self.counts = {}
        for total in range(12, 21):
            key = str(total) + '-0-0'
            self.counts[key] = [0, 0]
            key = str(total) + '-0-1'
            self.counts[key] = [0, 0]
            key = str(total) + '-1-0'
            self.counts[key] = [0, 0]
            key = str(total) + '-1-1'
            self.counts[key] = [0, 0]

    def incCount(self, key):
        self.counts[key][0] += 1

    def updProb(self, key, outcome):
        self.counts[key][1] += outcome 

In [39]:
class player:
    def __init__(self):
        self.cards = [] # cards currently in hand
        self.total = 0 # card total
        self.hasAce = False # True if an ace is being used as a 11
    
    def addCard(self, card):
        self.cards.append(card)
        if(card.rank == "J" or card.rank == "Q" or card.rank == "K"):
            self.total += 10
        elif(card.rank == "A" and self.hasAce == False):
            self.total += 11
            self.hasAce = True
        else:
            if card.rank == 'A' and self.hasAce: # you can not have 2 aces as 11, as that would make a total of 22
                self.total += 1 # use the ace as a one
            else:
                self.total += int(card.rank)
        
        if self.total > 21 and self.hasAce == True:
            self.hasAce = False
            self.total -= 10 # 11 - 10 = 1
    
    def hit(self, myDeck):
        self.addCard(myDeck.getRandomCard())
    
    
    def clear(self):
        self.cards = []
        self.total = 0
        self.hasAce = False

In [5]:
class card:
    def __init__(self, suit, rank):
        self.suit = suit # ♧, ♢, ♥, ♤
        self.rank = rank # 2, 3, 4, 5, 6, 7, 8, 9, 10, J, Q, K, A

In [6]:
class deck:
    def __init__(self):
        self.cards = []
        self.setDeck()
        
    def setDeck(self):
        suits = ["♧", "♢", "♥", "♤"]
        ranks = [2, 3, 4, 5, 6, 7, 8, 9, 10, "J", "Q", "K", "A"]

        for suit in suits:
            for rank in ranks:
                self.cards.append(card(suit, rank))  # init deck
    
    def printDeck(self):
        for card in self.cards:
            print(card.rank, card.suit)
    
    def shuffle(self):
        random.shuffle(self.cards) # change card order

    def getRandomCard(self):
        temp = self.cards[0]
        del self.cards[0] # delete a card form the deck
        return temp

In [7]:
class blackJackRound:
    def __init__(self, deck, player, dealer):
        self.player = player
        self.dealer = dealer
        self.deck = deck
        self.newRound()
        
    
    def newRound(self):
        self.player.clear() # init found
        self.dealer.clear()
        self.deck.setDeck()
        self.deck.shuffle()
        self.hitPlayerBool = True

        self.player.hit(self.deck) # give player 2 cards
        self.player.hit(self.deck)
        self.dealer.hit(self.deck) # give dealer 1 card
    
    def hitDealer(self):
        if(self.player.total <= 21): # dealer will only hit if player is not bust
            while(self.dealer.total < 17): # dealer will always hit till his total is greather than 17
                self.dealer.hit(self.deck)
    
    def hitPlayer(self):
        while(self.hitPlayerBool):
            if(self.player.total == 21):
                self.hitPlayerBool = False
            elif(self.player.total < 12):
                #Ai go hererererere
                self.player.hit(self.deck)
            else:
                self.hitPlayerBool = False
    
    def outcome(self): # print outcome
        print("Player: ", self.player.total)
        print("Dealer: ", self.dealer.total)

        if(self.player.total <= 21 and (self.player.total > self.dealer.total or self.dealer.total > 21)): # player wins
            print("Player wins")
        elif(self.player.total == self.dealer.total): # player and dealer same total
            print("Draw")
        else:
            print("Dealer wins")
    
    def playRound(self): # play full round
        self.newRound()
        self.hitPlayer()
        self.hitDealer()
        self.outcome()

In [8]:
class monteCarloControl:
    def __init__(self):
        self.tracker = stateActionVal()

    def exec(self, numLoop):
        for k in range((numLoop + 1)):
            continue

In [40]:
class SARSA:
    def __init__(self):
        self.stateActionVal = stateActionVal()
    
    def training(self, numOfEps):
        discountFactor = 1
        dealer = player()
        p = player()
        d = deck()
        b = blackJackRound(d, p, dealer)
    
        for k in range(1, numOfEps):
            b.newRound()
            e = 0.1
            while b.player.total < 12:
                b.player.hit(b.deck)
            
            if b.player.total != 21:
                sa = epsilonGreedy([b.player.total, b.player.hasAce], e, self.stateActionVal)
                learningFactor = 1/(self.stateActionVal.counts[sa][0] + 1)

            while b.player.total < 21:
                # Execute action an state s
                if sa[-1] == 1:
                    b.player.hit()
                else:
                    self.stateActionVal.counts[sa][0] += 1
                    break
                
                # Get reward n form s`
                if b.player.total < 22:
                    n = 10 - (21 - b.player.total)
                    # Get next action a`
                    a = epsilonGreedy([b.player.total, b.player.hasAce], e, self.stateActionVal)

                     # Get Q(s`, a`) from table
                    saPrime = self.stateActionVal[a][1] # 0
                
                    # update
                    self.stateActionVal.counts[sa][0] += 1
                    self.stateActionVal.counts[sa][1] = self.stateActionVal.counts[sa][1] + learningFactor*(n + discountFactor*saPrime - self.stateActionVal.counts[sa][1])
                else:
                    self.stateActionVal.counts[sa][1] -= 1

                #seting new sa
                sa = saPrime
            
            b.hitDealer()
            b.outcome()
            if(b.player.total <= 21 and (b.player.total > b.dealer.total or b.dealer.total > 21)):
                self.stateActionVal.counts[sa][1] += 1
            elif(b.player.total == b.dealer.total):
                self.stateActionVal.counts[sa][1] += 0
            elif(b.player.total <= 21):
                self.stateActionVal.counts[sa][1] -= 1



In [15]:
def epsilonGreedy(state, epsilonVal, stateAcVal): #state will be and array [total, hasAce]
        s1 = str(state[0])+"-"+str(int(state[1]))+"-0"
        s2 = str(state[0])+"-"+str(int(state[1]))+"-1"
        a1 = stateAcVal.counts[s1]
        a2 = stateAcVal.counts[s2]

        if a1[1] < a2[1]:
            worseOption = s1
            bestOption = s2
        else:
            worseOption = s2
            bestOption = s1

        if random.uniform(0, 1) < epsilonVal:
            return worseOption
        else:
            return bestOption

In [41]:
sarse = SARSA()

sarse.training(1000)

Player:  19
Dealer:  18
Player wins
Player:  19
Dealer:  19
Draw
Player:  17
Dealer:  19
Dealer wins
Player:  18
Dealer:  18
Draw
Player:  14
Dealer:  19
Dealer wins
Player:  19
Dealer:  17
Player wins
Player:  19
Dealer:  21
Dealer wins
Player:  14
Dealer:  20
Dealer wins
Player:  16
Dealer:  17
Dealer wins
Player:  13
Dealer:  20
Dealer wins
Player:  12
Dealer:  17
Dealer wins
Player:  21
Dealer:  18
Player wins
Player:  14
Dealer:  21
Dealer wins
Player:  16
Dealer:  20
Dealer wins
Player:  20
Dealer:  19
Player wins
Player:  16
Dealer:  19
Dealer wins
Player:  20
Dealer:  19
Player wins
Player:  18
Dealer:  20
Dealer wins
Player:  17
Dealer:  23
Player wins
Player:  15
Dealer:  18
Dealer wins
Player:  12
Dealer:  21
Dealer wins
Player:  19
Dealer:  26
Player wins
Player:  12
Dealer:  17
Dealer wins
Player:  12
Dealer:  17
Dealer wins
Player:  21
Dealer:  19
Player wins
Player:  19
Dealer:  26
Player wins
Player:  17
Dealer:  18
Dealer wins
Player:  15
Dealer:  23
Player wins
Player